In [2]:
import xarray as xr
import cftime
import numpy as np

fp = "/beegfs/CMIP6/jdpaul3/cmip6_regrid_for_rasdaman_test/cmip6_regrid_mon_ensemble.nc"

In [3]:
ds = xr.open_dataset(fp, decode_cf=True)
ds

<xarray.Dataset> Size: 25GB
Dimensions:      (model: 14, scenario: 5, time: 1812, lat: 43, lon: 288)
Coordinates:
  * model        (model) int64 112B 0 1 2 3 4 5 6 7 8 9 10 11 12 13
  * scenario     (scenario) int64 40B 0 1 2 3 4
  * lat          (lat) float64 344B 90.0 89.06 88.12 87.17 ... 52.3 51.36 50.42
  * lon          (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * time         (time) object 14kB 1950-01-15 12:00:00 ... 2100-12-15 12:00:00
Data variables:
    pr           (model, scenario, time, lat, lon) float64 13GB ...
    tas          (model, scenario, time, lat, lon) float32 6GB ...
    tasmin       (model, scenario, time, lat, lon) float32 6GB ...
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    title:        CMIP6 Monthly Data on a Common Grid with Multi-Model Ensemb...
    description:  Monthly data from 13 CMIP6 models on a common grid, includi...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    contact:      uaf-snap-data-tools@alaska.edu
    url:          https://uaf-snap.org/

In [4]:
# test each variable to make sure the ensemble mean is calculated correctly

# historical data

lat = 65
lon = -142
time = "2000-01-15"


scenario = 0
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Variable: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Variable: pr
Original Dataset Values:
[[29.97666952]
 [27.69984897]
 [21.15413271]
 [46.11822293]
 [42.40550041]
 [61.02628084]
 [28.64923733]
 [36.82085066]
 [42.6551992 ]
 [45.24297638]
 [40.70989748]
 [61.57741884]
 [28.62439754]
 [39.43543329]]

 Calculated Ensemble Mean:
39.43543329257339
Dataset Ensemble Mean:
[39.43543329]



Variable: tas
Original Dataset Values:
[[-20.429276 ]
 [-20.059967 ]
 [-20.119385 ]
 [-23.313309 ]
 [-13.495392 ]
 [-30.471466 ]
 [-22.086227 ]
 [-17.520966 ]
 [-15.841095 ]
 [-24.740646 ]
 [-14.8836975]
 [-29.804474 ]
 [-18.590149 ]
 [-20.873543 ]]

 Calculated Ensemble Mean:
-20.87354
Dataset Ensemble Mean:
[-20.873543]



Variable: tasmin
Original Dataset Values:
[[       nan]
 [-24.734146]
 [       nan]
 [-25.717834]
 [-16.007416]
 [-33.87674 ]
 [-26.175766]
 [-21.404907]
 [-19.15184 ]
 [-29.399902]
 [-18.127762]
 [       nan]
 [       nan]
 [-23.844034]]

 Calculated Ensemble Mean:
-23.844036
Dataset Ensemble Mean:
[-23.844034]





In [5]:
# test each variable to make sure the ensemble mean is calculated correctly

# projected data

lat = 65
lon = -142
time = "2050-01-15"


scenario = 1
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Variable: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Variable: pr
Original Dataset Values:
[[55.54462963]
 [39.66422415]
 [        nan]
 [14.99605319]
 [37.8559323 ]
 [12.83805905]
 [18.00234453]
 [15.3692908 ]
 [50.48366914]
 [25.58397919]
 [33.05992719]
 [73.24867371]
 [33.68493904]
 [34.19431016]]

 Calculated Ensemble Mean:
34.19431016154704
Dataset Ensemble Mean:
[34.19431016]



Variable: tas
Original Dataset Values:
[[-12.891937]
 [-23.289185]
 [       nan]
 [-25.368423]
 [-17.122437]
 [-19.99533 ]
 [-25.04013 ]
 [-31.729004]
 [-17.027863]
 [-20.927277]
 [-22.76506 ]
 [-15.304871]
 [-16.065735]
 [-20.627272]]

 Calculated Ensemble Mean:
-20.62727
Dataset Ensemble Mean:
[-20.627272]



Variable: tasmin
Original Dataset Values:
[[-24.375732]
 [-28.014389]
 [       nan]
 [-27.32608 ]
 [-21.621994]
 [-23.45813 ]
 [-28.589447]
 [-33.471695]
 [-19.9169  ]
 [-24.432693]
 [-26.526764]
 [       nan]
 [       nan]
 [-25.773382]]

 Calculated Ensemble Mean:
-25.773382
Dataset Ensemble Mean:
[-25.773382]



